# L2R in AceKG
---
- with the example in DDEKG

In [28]:
import pymysql
import ujson
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
np.set_printoptions(threshold = np.inf, suppress = True)

In [2]:
db = pymysql.connect(host="10.10.10.10", port=3306, user="readonly", password="readonly", db="acekg", charset="utf8")

## Data
---
- paper question id => qid (query id)

In [3]:
f_pqid = open('data/pqid.txt', 'r', encoding='utf8')
qid = {}
flag = 0
for line in f_pqid:
    c = line[:-1].replace("\"","").split('\t')
    qid[str(c[0]) + "@" + str(c[1])] = flag
    flag += 1

In [4]:
qid

{'433977@method': 0,
 '433977@one_sentence_summary': 1,
 '433977@problem': 2,
 '527599@method': 3,
 '527599@one_sentence_summary': 4,
 '527599@performance_improvements': 5,
 '527599@problem': 6,
 '527599@the_method_is_based_on': 7,
 '1036524@experiment': 8,
 '1036524@method': 9,
 '1036524@one_sentence_summary': 10,
 '1036524@the_deferences_comparing_to_previous_methods': 11,
 '1036524@the_method_is_based_on': 12,
 '1227799@aim': 13,
 '1227799@method': 14,
 '1227799@methor_base': 15,
 '1227799@problem': 16,
 '2488495@experiment': 17,
 '2488495@method': 18,
 '2488495@one_sentence_summary': 19,
 '2488495@performance_improvements': 20,
 '2488495@problem': 21,
 '2488495@results': 22,
 '2488495@the_deferences_comparing_to_previous_methods': 23,
 '2488495@the_method_is_based_on': 24,
 '3032562@aim': 25,
 '3032562@experiment': 26,
 '3032562@method': 27,
 '3032562@methor_base': 28,
 '3032562@problem': 29,
 '3032562@result_outperform': 30,
 '3297940@aim': 31,
 '3297940@result': 32,
 '3558404@aim

### Benchmark
---
- 52 absolutely correct top3 entity

In [5]:
f_benchmark_paper_id = open('data/benchmark_paper.txt', 'r', encoding='utf8')
f_benchmark_res_top3 = open('data/benchmark_result.txt', 'r', encoding='utf8')

bm_paper = []
for line in f_benchmark_paper_id:
    bm_paper.append(line[:-1])

bm_answer = []
for line in f_benchmark_res_top3:
    bm_answer.append(line[:-1])

In [6]:
bm_paper[:2], bm_answer[:2]

(['420342715', '87250949'], ['53641', '53642'])

> abs_score, title_score, qa_score, word_len, letter_len, complex_len

In [7]:
def get_benchmark(paper_id):
    cur = db.cursor()
    sql='SELECT paper_id, question, abs_score, title_score, qa_score, word_len, letter_len, complex_len, dpaqn_id FROM dde_paper_abstract_QA_ner WHERE paper_id = ' + str(paper_id)
    cur.execute(sql)
    one_paper_bm = []
    try:
        result = cur.fetchall()
        for res in result:
            para = []
            for i in range(2, 8):
                para.append(round(float(res[i]),5))
            one_line_bm = "qid:"+str(qid[str(res[0])+"@"+str(res[1])])+" 1:"+str(round(para[0], 5))+" 2:"+str(round(para[1], 5))+" 3:"+str(round(para[2], 5))+" 4:"+str(round(para[3], 5))+" 5:"+str(round(para[4], 5))+" 6:"+str(round(para[5], 5))
            one_paper_bm.append((one_line_bm, res[8]))
        return one_paper_bm
    except:
        print(paper_id)
        return one_paper_bm

In [9]:
get_benchmark(420342715)

[('qid:5143 1:0.16046 2:0.22299 3:0.36968 4:2.0 5:17.0 6:1.0', 29104),
 ('qid:5143 1:0.13081 2:0.14377 3:0.32 4:2.0 5:17.0 6:1.0', 29105),
 ('qid:5143 1:0.09939 2:0.26934 3:0.18308 4:1.0 5:10.0 6:0.0', 29106),
 ('qid:5143 1:0.12657 2:0.10073 3:0.3283 4:1.0 5:7.0 6:0.0', 29107),
 ('qid:5143 1:0.12446 2:0.25913 3:0.17614 4:1.0 5:9.0 6:0.0', 29108),
 ('qid:5143 1:0.10691 2:0.2087 3:0.14186 4:1.0 5:9.0 6:0.0', 29109),
 ('qid:5143 1:0.11803 2:0.15885 3:0.10798 4:1.0 5:11.0 6:0.0', 29110),
 ('qid:5143 1:0.11565 2:0.03756 3:0.2475 4:1.0 5:6.0 6:0.0', 29111),
 ('qid:5143 1:0.06985 2:0.02143 3:0.15978 4:1.0 5:10.0 6:0.0', 29112),
 ('qid:5143 1:0.0476 2:0.15894 3:0.10804 4:1.0 5:6.0 6:0.0', 29113),
 ('qid:5143 1:0.06633 2:0.0 3:0.17993 4:1.0 5:6.0 6:0.0', 29114),
 ('qid:5143 1:0.06622 2:0.00668 3:0.13571 4:1.0 5:6.0 6:0.0', 29115),
 ('qid:5143 1:0.04 2:0.10401 3:0.0707 4:2.0 5:14.0 6:1.0', 29116),
 ('qid:5143 1:0.00905 2:0.0 3:0.02454 4:1.0 5:9.0 6:0.0', 29117),
 ('qid:5143 1:0.02471 2:0.0 3:0.0

In [10]:
benchmark_tmp = []
for pid in tqdm(bm_paper):
    benchmark_tmp += get_benchmark(paper_id=pid)

In [11]:
benchmark = []
dp_bench = []
for tmp in benchmark_tmp:
    if str(tmp[1]) in bm_answer:
        bm_tmp = '1 ' + tmp[0]
    else:
        bm_tmp = '0 ' + tmp[0]
    benchmark.append(bm_tmp)
    dp_bench.append(tmp[1])

In [12]:
benchmark[:10], dp_bench[:10]

(['1 qid:5143 1:0.16046 2:0.22299 3:0.36968 4:2.0 5:17.0 6:1.0',
  '1 qid:5143 1:0.13081 2:0.14377 3:0.32 4:2.0 5:17.0 6:1.0',
  '0 qid:5143 1:0.09939 2:0.26934 3:0.18308 4:1.0 5:10.0 6:0.0',
  '0 qid:5143 1:0.12657 2:0.10073 3:0.3283 4:1.0 5:7.0 6:0.0',
  '0 qid:5143 1:0.12446 2:0.25913 3:0.17614 4:1.0 5:9.0 6:0.0',
  '0 qid:5143 1:0.10691 2:0.2087 3:0.14186 4:1.0 5:9.0 6:0.0',
  '0 qid:5143 1:0.11803 2:0.15885 3:0.10798 4:1.0 5:11.0 6:0.0',
  '0 qid:5143 1:0.11565 2:0.03756 3:0.2475 4:1.0 5:6.0 6:0.0',
  '0 qid:5143 1:0.06985 2:0.02143 3:0.15978 4:1.0 5:10.0 6:0.0',
  '0 qid:5143 1:0.0476 2:0.15894 3:0.10804 4:1.0 5:6.0 6:0.0'],
 [29104, 29105, 29106, 29107, 29108, 29109, 29110, 29111, 29112, 29113])

In [13]:
bm = open('data/benchmark.txt', 'w', encoding='utf8')
for line in benchmark:
    print(line[:-1], file=bm)

In [14]:
bm.close()
f_benchmark_paper_id.close()
f_benchmark_res_top3.close()
f_pqid.close()

### train&test set
---
- collect by [mark.acemap.cn](mark.acemap.cn)
- **including the benchamark**, The first column is label, the second column is qid, and the following columns are features (total 6 features).

In [15]:
def get_dataset():
    dataset = []
    dp_list = []
    cur = db.cursor()
    sql='SELECT paper_id, question, abs_score, title_score,qa_score, word_len, letter_len, complex_len, mark, dpaqn_id FROM dde_paper_abstract_QA_ner where mark is not null'
    cur.execute(sql)
    result = cur.fetchall()
    for res in result:
        para = []
        for i in range(2, 8):
            para.append(round(float(res[i]),5))
        one_line = str(res[8])+" qid:"+str(qid[str(res[0])+"@"+str(res[1])])+" 1:"+str(round(para[0], 5))+" 2:"+str(round(para[1], 5))+" 3:"+str(round(para[2], 5))+" 4:"+str(round(para[3], 5))+" 5:"+str(round(para[4], 5))+" 6:"+str(round(para[5], 5))
        dataset.append(one_line)
        dp_list.append(int(res[9]))
    return dataset, dp_list

In [16]:
dataset, dp_list = get_dataset()
len(dataset)

8302

In [17]:
ds = open('data/all.txt', 'w', encoding='utf8')

for line in dataset:
    print(line[:-1], file=ds)
ds.close()

In [18]:
db.close()

- divide into train:test = 4:1

In [19]:
dataset[1]

'0 qid:4073 1:0.02323 2:0.02874 3:0.04246 4:1.0 5:10.0 6:0.0'

In [24]:
def build_dateset(dataset, dp_list):
    training_set = []
    testing_set = []
    dp_train = []
    dp_test = []
    for index in range(0, len(dataset)):
        c = dataset[index].split(" ")
        record = [int(c[0]), float(c[1].split(":")[1]), float(c[2].split(":")[1]), float(c[3].split(":")[1]), float(c[4].split(":")[1]), float(c[5].split(":")[1]), float(c[6].split(":")[1]), float(c[7].split(":")[0])]    
        if index < 4/5*len(dataset):
            training_set.append(record)
            dp_train.append(dp_list[index])
        else:
            testing_set.append(record)
            dp_test.append(dp_list[index])
    return training_set, testing_set, dp_train, dp_test

def build_benchmark(benchmark):
    benchmark_set = []
    for index in range(0, len(dataset)):
        c = dataset[index].split(" ")
        record = [int(c[0]), float(c[1].split(":")[1]), float(c[2].split(":")[1]), float(c[3].split(":")[1]), float(c[4].split(":")[1]), float(c[5].split(":")[1]), float(c[6].split(":")[1]), float(c[7].split(":")[0])]    
        benchmark_set.append(record)
    return benchmark_set

In [25]:
training_set, testing_set, dp_train, dp_test = build_dateset(dataset, dp_list)
benchmark_set = build_benchmark(benchmark)

In [29]:
training_set[:3]
trs = np.array(training_set)
tes = np.array(testing_set)
trs[:3], tes[:3]

(array([[   1.     , 4073.     ,    0.07169,    0.11169,    0.16501,
            1.     ,    8.     ,    6.     ],
        [   0.     , 4073.     ,    0.02323,    0.02874,    0.04246,
            1.     ,   10.     ,    6.     ],
        [   0.     , 4073.     ,    0.03952,    0.0438 ,    0.06472,
            1.     ,    9.     ,    6.     ]]),
 array([[   0.     , 4700.     ,    0.     ,    0.     ,    0.31876,
            1.     ,   12.     ,    6.     ],
        [   0.     , 4700.     ,    0.02548,    0.     ,    0.03204,
            1.     ,    3.     ,    6.     ],
        [   0.     , 4700.     ,    0.02409,    0.     ,    0.03135,
            1.     ,    3.     ,    6.     ]]))

In [30]:
bms = np.array(benchmark_set)
bms[:3]

array([[   1.     , 4073.     ,    0.07169,    0.11169,    0.16501,
           1.     ,    8.     ,    6.     ],
       [   0.     , 4073.     ,    0.02323,    0.02874,    0.04246,
           1.     ,   10.     ,    6.     ],
       [   0.     , 4073.     ,    0.03952,    0.0438 ,    0.06472,
           1.     ,    9.     ,    6.     ]])

In [31]:
np.save('train.npy', trs)
np.save('test.npy', tes)
np.save('benchmark.npy', bms)

## L2R Model

In [32]:
import LambdaRank as LR
import LambdaMART as LM
import RankNet as RN

In [47]:
train_data = np.load("data/train.npy")
test_data = np.load("data/test.npy")

In [59]:
train_data.shape

(9630, 48)

### RankNet

In [33]:
n_feature = 46
h1_units = 512
h2_units = 256
epoch = 100
learning_rate = 0.01
plot=False

- **train**

In [64]:
model = RN.RankNet(n_feature, h1_units, h2_units, epoch, learning_rate, plot)
model.fit(train_data)

Traning………………

Epoch:0, loss : 0.6975727677345276
Epoch:10, loss : 0.44026926159858704
Epoch:20, loss : 0.391196608543396
Epoch:30, loss : 0.35077837109565735
Epoch:40, loss : 0.3178143799304962
Epoch:50, loss : 0.28842833638191223
Epoch:60, loss : 0.2596825063228607
Epoch:70, loss : 0.24030372500419617
Epoch:80, loss : 0.21682649850845337
Epoch:90, loss : 0.19929306209087372


- **test**

In [68]:
k = 4
model.validate(test_data, k)

Average NDCG@4 is 0.5531070145386872


### LambdaRank

In [34]:
n_feature = trs.shape[1] - 2
h1_units = 512
h2_units = 256
epoch = 10
learning_rate = 0.0001

**train**

In [35]:
model = LR.LambdaRank(trs, n_feature, h1_units, h2_units, epoch, learning_rate)
model.fit()

Training .....

Epoch:0, Average NDCG : 0.6953825597990645
Epoch:1, Average NDCG : 0.6832248339209323
Epoch:2, Average NDCG : 0.6970276446591956
Epoch:3, Average NDCG : 0.703493484112893
Epoch:4, Average NDCG : 0.7029519874232826
Epoch:5, Average NDCG : 0.705372319072394
Epoch:6, Average NDCG : 0.7055624456250165
Epoch:7, Average NDCG : 0.7070044003639552
Epoch:8, Average NDCG : 0.70759167896587
Epoch:9, Average NDCG : 0.7094013936052286


**test**

In [96]:
k = 3
ndcg = model.validate(tes, k)
print(ndcg)

[nan, 0.9291451193271983, 0.9374130297304558, 0.9070356189982551, 0.7737459235132744, 0.6325004241086445, 0.519301112470348, 0.7461976723150332, 1.0, 0.5, nan, 0.8315546295836225, 0.9072602453428914, 0.9740298100854945, 0.46845052016107697, 0.9469024295259745, nan, 0.5204370438439433, nan, 0.6309297535714575, 1.0, 0.9197207891481876, nan, 1.0, nan, nan, 0.6309297535714575, nan, 0.38685280723454163, 0.8665427526082052, 0.8665427526082052, 0.6509209298071326, 1.0, 0.511558309303655, 0.7841802768331765, nan, 1.0, nan, 0.5, 0.3562071871080222, 1.0, nan, 1.0, 1.0, 0.3010299956639812, nan, 0.3333333333333333, nan, nan, 1.0, nan, nan, nan, 0.5437713091520254, 0.8517029262244226, 0.39564672360221187, 0.5626358745665102, 0.5833416105149332, nan, nan, 0.5127311674319409, 0.38685280723454163, 0.6309297535714575, 0.5, nan, 0.6509209298071326, 0.8175295903539445, 0.8838242945899706, 0.9558295932317544, 0.7565030690421024, 0.7827141072039429, 0.8009452130544324, 0.5239472913541398, 0.430676558073393

In [43]:
LR_res = model.predict(tes)
for i in range(0, len(LR_res)):
    print(LR_res[i], testing_set[i][0:2], dp_test[i])

1.5906283855438232 [0, 4700.0] 51892
1.0665507316589355 [0, 4700.0] 51893
1.0662977695465088 [0, 4700.0] 51894
1.5513052940368652 [0, 4700.0] 51895
1.3895046710968018 [0, 4700.0] 51896
1.0906751155853271 [0, 4700.0] 51897
1.2480907440185547 [0, 4700.0] 51898
2.162419319152832 [1, 4698.0] 51899
1.687035083770752 [1, 4698.0] 51900
1.3796696662902832 [1, 4698.0] 51901
1.3785674571990967 [1, 4698.0] 51902
1.2786550521850586 [0, 4698.0] 51903
1.4864566326141357 [0, 4698.0] 51904
1.2714858055114746 [0, 4698.0] 51905
1.1510593891143799 [1, 4698.0] 51906
1.3399724960327148 [1, 4698.0] 51907
2.095144748687744 [1, 4698.0] 51908
1.1951158046722412 [0, 4698.0] 51909
1.5508837699890137 [0, 4698.0] 51910
1.3891983032226562 [0, 4698.0] 51911
1.4922571182250977 [1, 4698.0] 51912
1.0906751155853271 [0, 4698.0] 51913
1.247602939605713 [0, 4698.0] 51914
2.1943089962005615 [1, 4699.0] 51915
1.3875160217285156 [1, 4699.0] 51916
1.6930112838745117 [1, 4699.0] 51917
1.293266773223877 [0, 4699.0] 51918
1.4167

### LambdaMART

In [14]:
number_of_trees = 20
lr = 0.01

In [16]:
model = LM.LambdaMART(train_data, number_of_trees, lr)
model.fit()

Tree 0
Epoch:0, Average NDCG : 0.9960848183140814
Tree 1
Epoch:1, Average NDCG : 0.9960848183140814
Tree 2
Epoch:2, Average NDCG : 0.9962218255813996
Tree 3
Epoch:3, Average NDCG : 0.9962218255813996
Tree 4
Epoch:4, Average NDCG : 0.9962218255813996
Tree 5
Epoch:5, Average NDCG : 0.9962218255813996
Tree 6
Epoch:6, Average NDCG : 0.9962218255813996
Tree 7
Epoch:7, Average NDCG : 0.9962218255813996
Tree 8
Epoch:8, Average NDCG : 0.9962218255813996
Tree 9
Epoch:9, Average NDCG : 0.9962218255813996
Tree 10
Epoch:10, Average NDCG : 0.9962218255813996
Tree 11
Epoch:11, Average NDCG : 0.9962218255813996
Tree 12
Epoch:12, Average NDCG : 0.9962218255813996
Tree 13
Epoch:13, Average NDCG : 0.9962218255813996
Tree 14
Epoch:14, Average NDCG : 0.9962218255813996
Tree 15
Epoch:15, Average NDCG : 0.9962218255813996
Tree 16
Epoch:16, Average NDCG : 0.9962218255813996
Tree 17
Epoch:17, Average NDCG : 0.9962218255813996
Tree 18
Epoch:18, Average NDCG : 0.9962218255813996
Tree 19
Epoch:19, Average NDCG :

In [17]:
k = 4
ndcg = model.validate(test_data, k)
print(ndcg)

[0.0, 0.20320665000263277, 1.0, 0.0, 0.6096199500078984, 0.4671445995090871, 0.52129602861432, nan, 0.7959761332981399, 0.0, nan, 0.0, 0.6366824387328317, nan, 0.7653606369886217, 0.7958830702855284, 0.6366824387328317, 0.6131471927654584, nan, nan, 0.52129602861432, 0.0, 1.0, 0.5307212739772434, 0.75369761125927, 1.0, 0.497223749750845, 0.16812753627111748, 0.5585075862632192, nan, 0.6131471927654584, nan, nan, 0.0, 0.0, 0.8472668887613066, 0.5, 0.6131471927654584, 0.8318724637288825, 0.3903800499921017, nan, nan, 0.5158474921363962, nan, 1.0, 0.8698733166692995, 0.5, 0.6713860725233041, nan, 0.11699506935449569, 0.2479235769830519, 0.17376534287144002, 0.6309297535714575, 0.42501747489323527, 0.6863656220351223, nan, nan, nan, nan, 1.0, 0.0, nan, nan, 0.8453659008195946, 0.6565619062195633, 0.75369761125927, 0.7017457803981819, nan, 0.6309297535714575, nan, 0.3868528072345416, 0.43067655807339306, 0.0, nan, nan, 0.4935456744811716, nan, 0.36331756126716835, nan, 0.0, 0.83187246372888

In [20]:
test_data

array([[0.00000e+00, 1.82190e+04, 5.28930e-02, ..., 3.96910e-01,
        4.47368e-01, 9.66667e-01],
       [0.00000e+00, 1.82190e+04, 4.95900e-03, ..., 1.92550e-02,
        4.21053e-01, 0.00000e+00],
       [0.00000e+00, 1.82190e+04, 6.61160e-02, ..., 4.51770e-02,
        4.73684e-01, 1.00000e+00],
       ...,
       [1.00000e+00, 1.99970e+04, 2.17950e-02, ..., 7.96000e-04,
        1.25000e-01, 1.00000e+00],
       [0.00000e+00, 1.99970e+04, 7.54270e-02, ..., 5.60000e-05,
        2.50000e-01, 4.73684e-01],
       [0.00000e+00, 1.99970e+04, 3.24790e-02, ..., 4.00000e-05,
        1.25000e-01, 0.00000e+00]])

In [19]:
res = model.predict(test_data)

[0.006425987526129728,
 0.048286688623789785,
 0.008731911601397956,
 -0.020275381351036854,
 0.09224557644666355,
 -0.021552023009594353,
 -0.007567350878894898,
 -0.017208220140600538,
 -0.034914500350316874,
 -0.05942122485456207,
 -0.009063140617031052,
 0.013859653852691056,
 0.011321927027433426,
 -0.0188096259575924,
 0.02263286467438871,
 -0.005780013902121664,
 -0.01080406830054825,
 -0.002585981376692355,
 0.11134477025682833,
 -0.011406979755574668,
 -0.005933148624141416,
 0.03304546841913483,
 -0.009100424863446782,
 -0.004152902474064401,
 -0.0024075444393255757,
 0.021730777822245768,
 0.06881199613816778,
 0.015424056756350803,
 0.012787714749505234,
 -0.009784585502931006,
 -0.014097963099257118,
 -0.006741735289088399,
 -0.003597326746482755,
 -0.009667539617345138,
 -0.012426235842595737,
 0.045659270627192,
 0.002335106728425784,
 -0.009765795525506573,
 0.010277919949918338,
 -0.0007477595851043174,
 -0.014097963099257118,
 -0.013532574020899314,
 -0.00053702715838